# Plot Dark image statistics and Masks

In [ ]:
import numpy as np, matplotlib.pyplot as plt, pandas as pd, datashader as ds

from pathlib import Path
from astropy.io import fits
from scipy import stats
from scipy.ndimage.morphology import binary_closing
from functools import partial
from mpl_ext import dsshow

In [ ]:
%matplotlib widget

In [ ]:
cuts = pd.read_csv("cuts.txt", delim_whitespace = True)

In [ ]:
data_path = "../../"

# Plot Stats

In [ ]:
#set camera name
camera = "B5"

In [ ]:
#read fits file into data frame
fitsfile = fits.open(data_path + "dark-stat-" + camera + ".fits")
median_img = fitsfile["MEDIAN"].data
iqr_img = fitsfile["IQR"].data

#ravel data and fix byte order
median = median_img.ravel().newbyteorder().byteswap()
iqr = iqr_img.ravel().newbyteorder().byteswap()

df = pd.DataFrame({"median": median, "iqr": iqr})

In [ ]:
#Fast, interactive scatter plot of the pixel values along with the thresholds
fig, ax = plt.subplots(1,1, figsize=(10,5))

dyn = partial(ds.tf.dynspread, max_px=40, threshold=0.5)

da = dsshow(df, ds.Point('median', 'iqr'), spread_fn=dyn, aspect="auto", ax=ax)

cut = cuts[cuts["camera"]==camera]
med_min = float(cut["med_min"])
med_max = float(cut["med_max"])
iqr_min = float(cut["iqr_min"])
iqr_max = float(cut["iqr_max"])

plt.axvline(med_min, color="k", ls="--")
plt.axvline(med_max, color="k", ls="--")
plt.axhline(iqr_min, color="k", ls="--")
plt.axhline(iqr_max, color="k", ls="--")

plt.xlabel("Median")
plt.ylabel("IQR")
plt.colorbar(da)

In [ ]:
#Uncomment if you want to plot axes in percentiles

# #convert ticks to percentile scale
# x_ticks = ax.get_xticks()
# x_percs = [np.round(stats.percentileofscore(median, p),4) for p in x_ticks]
# # ax.set_xticks(x_ticks)
# ax.set_xticklabels(x_percs)

# y_ticks = ax.get_yticks()
# y_percs = [np.round(stats.percentileofscore(median, p),4) for p in y_ticks]
# # ax.set_yticks(y_ticks)
# ax.set_yticklabels(y_percs)
# plt.title("Camera: B0", size=20)

In [ ]:
#Cuts in percentile
minmed=stats.percentileofscore(median, med_min)
print("Min Median:", minmed)
maxmed=stats.percentileofscore(median, med_max)
print("Max Median:", maxmed)
miniqr=stats.percentileofscore(iqr, iqr_min)
print("Min IQR:", miniqr)
maxiqr=stats.percentileofscore(iqr, iqr_max)
print("Max IQR:", maxiqr)

# Generate  and Plot Mask

In [ ]:
def create_mask(median_img, iqr_img, minmed, maxmed, miniqr, maxiqr, colfrac=0.4, closeiter=20):
    #Create the masks
    mask   = np.zeros(median_img.shape, dtype=np.int32)
    
    minmed = np.percentile(median_img, minmed)
    maxmed = np.percentile(median_img, maxmed)
    miniqr = np.percentile(iqr_img, miniqr)
    maxiqr = np.percentile(iqr_img, maxiqr)
    
    #Set the Bad flag absed on thresholds
    mask[(median_img>maxmed)|(median_img<minmed)|(iqr_img>maxiqr)|(iqr_img<miniqr)] |= 1
    
    #Close incompletely blocked regions
    closed_mask = binary_closing(mask, iterations=closeiter, structure=np.ones([2,2]).astype(np.int32)) #returns binary array
    closed_mask[closed_mask] = 1
    mask |= closed_mask
    
    #Block entire columns above a certain threshold per amplifier
    bad_pix = (mask>0)
    bad_pix_upper = bad_pix[0:bad_pix.shape[0]//2,:]
    bad_pix_lower = bad_pix[bad_pix.shape[0]//2:bad_pix.shape[0],:]
    bad_frac_upper = np.sum(bad_pix_upper, axis=0)/(bad_pix.shape[0]//2)
    bad_frac_lower = np.sum(bad_pix_lower, axis=0)/(bad_pix.shape[0]//2)
    bad_cols_upper = np.where(bad_frac_upper>=colfrac)
    bad_cols_lower = np.where(bad_frac_lower>=colfrac)
    mask[0:bad_pix.shape[0]//2,bad_cols_upper] |= 1
    mask[bad_pix.shape[0]//2:bad_pix.shape[0],bad_cols_lower] |= 1
    
    return mask

### Plot Mask

In [ ]:
# mask_img = fits.open(data_path + "pixmask-" + camera + ".fits")[0].data
mask_img = create_mask(median_img, iqr_img, minmed, maxmed, miniqr, maxiqr, closeiter=20 )

print("Blocked Percent:", np.sum(mask_img>0)*100/np.product(mask_img.shape))

In [ ]:
plt.figure(figsize=(12,12))
plt.imshow(mask_img, cmap="binary", vmin=0, vmax=1, origin="lower" )
plt.grid()
plt.show()